<h1> RQ4 </h1>
This code is written by Santo Palaia and Pooja Jambaladinni.

In [1]:
#import libraries
import json
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Load the json files
event = pd.read_json(r'/users/poojacj/Desktop/adm_hw2/events/events_England.json')[['eventId', 'tags', 'playerId','matchId']]
#consider only pass
pass_event_all=event.loc[event['eventId']==8] 

In [3]:
pass_event_all

,eventId,tags,playerId,matchId
0,8,[{'id': 1801}],25413,2499719
1,8,[{'id': 1801}],370224,2499719
2,8,[{'id': 1801}],3319,2499719
3,8,[{'id': 1801}],120339,2499719
4,8,[{'id': 1801}],167145,2499719
...,...,...,...,...
643134,8,[{'id': 1801}],8313,2500098
643135,8,[{'id': 1801}],379209,2500098
643141,8,[{'id': 1801}],8561,2500098
643142,8,[{'id': 1801}],20620,2500098


In [4]:
#drop column eventId
pass_event_all.drop('eventId', axis=1, inplace=True) #we don't need eventId

In [5]:
myList=[]
for index, row in pass_event_all.iterrows():
    myList.append(1) if {'id':1801} in row['tags'] else myList.append(0) #create a list of 1 (pass compleate) and 0 (pass not totaly complete)

In [6]:
pass_event_all['pass']=myList #add this column
pass_event_all.drop('tags', axis=1, inplace=True) #we don't need tags

In [7]:
pass_event_all

,playerId,matchId,pass
0,25413,2499719,1
1,370224,2499719,1
2,3319,2499719,1
3,120339,2499719,1
4,167145,2499719,1
...,...,...,...
643134,8313,2500098,1
643135,379209,2500098,1
643141,8561,2500098,1
643142,20620,2500098,1


In [8]:
#choosing the threshold value for minimum attempted passes by calculating the average passes attempted for a match per player
df_threshold = (pass_event_all.groupby(['playerId', 'matchId'], as_index=False).count()
            .groupby('playerId')['pass'].mean())

In [9]:
threshold_value = df_threshold.mean()

In [10]:
pass_event_final=pass_event_all.groupby('playerId').agg({'pass':['sum', 'count']}) #grouping on playerId and aggregating on sum(number of comlete pass) and count of total pass tried
pass_event_final.columns=['_'.join(col).strip() for col in pass_event_final.columns.values] #redefination of columns
pass_event_final=pass_event_final.reset_index() #reset the index

In [11]:
pass_event_final

,playerId,pass_sum,pass_count
0,0,0,6
1,36,790,898
2,38,192,224
3,48,2113,2370
4,54,1803,2196
...,...,...,...
509,448708,5,6
510,450826,12,17
511,486252,165,226
512,531655,7,11


In [12]:
#filtering out the data on threshold value
pass_event_final = pass_event_final[pass_event_final["pass_count"] > threshold_value]

In [13]:
pass_event_final

,playerId,pass_sum,pass_count
1,36,790,898
2,38,192,224
3,48,2113,2370
4,54,1803,2196
5,56,78,97
...,...,...,...
502,442211,62,81
503,447205,38,43
505,447379,69,84
507,447812,126,171


In [14]:
pass_event_final["ratio"] = pass_event_final["pass_sum"] / pass_event_final["pass_count"]

In [15]:
pass_event_final

,playerId,pass_sum,pass_count,ratio
1,36,790,898,0.879733
2,38,192,224,0.857143
3,48,2113,2370,0.891561
4,54,1803,2196,0.821038
5,56,78,97,0.804124
...,...,...,...,...
502,442211,62,81,0.765432
503,447205,38,43,0.883721
505,447379,69,84,0.821429
507,447812,126,171,0.736842


In [16]:
#sort the columns in descending order of ratio
pass_event_final_sorted = pass_event_final.sort_values(by='ratio', ascending=False)

In [17]:
pass_event_final_sorted.head(15)

,playerId,pass_sum,pass_count,ratio
223,9380,1175,1204,0.975914
142,8324,383,404,0.948020
159,8464,1497,1584,0.945076
137,8307,1312,1391,0.943206
401,135103,1211,1288,0.940217
59,7856,219,233,0.939914
77,7918,992,1057,0.938505
356,70086,2779,2964,0.937584
391,120339,781,834,0.936451
253,14808,1263,1350,0.935556


In [18]:
#select the top 10 rows with maximun value of ratio
df_top_players = pass_event_final_sorted[:10]

In [19]:
df_top_players

,playerId,pass_sum,pass_count,ratio
223,9380,1175,1204,0.975914
142,8324,383,404,0.948020
159,8464,1497,1584,0.945076
137,8307,1312,1391,0.943206
401,135103,1211,1288,0.940217
59,7856,219,233,0.939914
77,7918,992,1057,0.938505
356,70086,2779,2964,0.937584
391,120339,781,834,0.936451
253,14808,1263,1350,0.935556


In [20]:
#select only the player ID and the ratio of passes
df_result = df_top_players[['playerId','ratio']]

In [21]:
#change the names of columns
df_result.columns = ['wyId','RatioPasses']

In [22]:
df_result

,wyId,RatioPasses
223,9380,0.975914
142,8324,0.948020
159,8464,0.945076
137,8307,0.943206
401,135103,0.940217
59,7856,0.939914
77,7918,0.938505
356,70086,0.937584
391,120339,0.936451
253,14808,0.935556


In [23]:
#load the player data into a dataframe
df_player = pd.read_json('/users/poojacj/Desktop/adm_hw2/players.json')[['lastName','wyId']]

In [24]:
df_player

,lastName,wyId
0,Tekin,32777
1,Sarr,393228
2,Mandanda,393230
3,N'Diaye,32793
4,Konat\u00e9,393247
...,...,...
3598,Ma\u00e2loul,120839
3599,C\u00e1ceda Oyaguez,114736
3600,Araujo Blanco,114908
3601,Tagnaouti,285583


In [25]:
#left join the teams and event results dataframes on id to get the names of players
df_merge_col = pd.merge(df_result, df_player, on='wyId', how='left')

In [26]:
df_merge_col

,wyId,RatioPasses,lastName
0,9380,0.975914,Stones
1,8324,0.948020,Tour\u00e9
2,8464,0.945076,Delph
3,8307,0.943206,Kompany
4,135103,0.940217,Christensen
5,7856,0.939914,Mertesacker
6,7918,0.938505,Jones
7,70086,0.937584,Otamendi
8,120339,0.936451,Elsayed Elneny
9,14808,0.935556,G\u00fcndo\u011fan


In [75]:
a = 'Tour\u00e9'
df_merge_col['lastName'][1] = a.encode('utf-8').decode('utf-8')

In [76]:
b = 'G\u00fcndo\u011fan'
df_merge_col['lastName'][9] = b.encode('utf-8').decode('utf-8')

In [77]:
df_merge_col['lastName']

0            Stones
1             Touré
2             Delph
3           Kompany
4       Christensen
5       Mertesacker
6             Jones
7          Otamendi
8    Elsayed Elneny
9          Gündoğan
Name: lastName, dtype: object

In [359]:
#save the results to a csv file
df_merge_col.to_csv("/users/poojacj/Desktop/adm_hw2/RQ4/highest_ratio_of_passes.csv",index=None, encoding='utf-16')